In [2]:
#from curl_cffi.requests import Session
# import yfinance as yf
import pandas as pd
import numpy as np
# import ffn
#from langchain_core.tools import tool
#from duckduckgo_search import DDGS
from openai import OpenAI
import os
import html
import urllib.parse
import requests
from bs4 import BeautifulSoup
#from googlesearch import search  # google 라이브러리 사용
import requests
from bs4 import BeautifulSoup
import os
#from pytrends.request import TrendReq
#from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import contextlib
import requests
from urllib3.exceptions import InsecureRequestWarning

old_merge_environment_settings = requests.Session.merge_environment_settings
@contextlib.contextmanager
def no_ssl_verification():
    opened_adapters = set()

    def merge_environment_settings(self, url, proxies, stream, verify, cert):
        # Verification happens only once per connection so we need to close
        # all the opened adapters once we're done. Otherwise, the effects of
        # verify=False persist beyond the end of this context manager.
        opened_adapters.add(self.get_adapter(url))

        settings = old_merge_environment_settings(self, url, proxies, stream, verify, cert)
        settings['verify'] = False

        return settings

    requests.Session.merge_environment_settings = merge_environment_settings

    try:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', InsecureRequestWarning)
            yield
    finally:
        requests.Session.merge_environment_settings = old_merge_environment_settings

        for adapter in opened_adapters:
            try:
                adapter.close()
            except:
                pass

#from transformers import pipeline
#with no_ssl_verification():
#    pipe = pipeline("text-classification", model="snunlp/KR-FinBert-SC")

In [3]:
load_dotenv()
client_id = os.getenv("NAVER_CLIENT_ID")
client_secret = os.getenv("NAVER_CLIENT_SECRET")

In [3]:
import os
import requests
import urllib.parse
import html
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
client_id = os.getenv("NAVER_CLIENT_ID")
client_secret = os.getenv("NAVER_CLIENT_SECRET")

In [5]:
import os
import requests
import urllib.parse
import html
from bs4 import BeautifulSoup
from datetime import datetime

def get_news_list(company_name: str, display=10) -> list:
    client_id = os.getenv("NAVER_CLIENT_ID")  # 환경 변수에서 클라이언트 ID 가져오기
    client_secret = os.getenv("NAVER_CLIENT_SECRET")  # 환경 변수에서 클라이언트 Secret 가져오기
    
    # 쿼리 URL 인코딩
    query = urllib.parse.quote(company_name)
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display={display}&sort=date"
    
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }

    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
        return []
    
    print("API 호출 성공!")
    items = response.json().get("items", [])
    
    if not items:
        print(f"❗ '{company_name}' 관련 뉴스가 없습니다.")
        return []

    # 뉴스 기사 내용 정제 함수
    def clean(text):
        return html.unescape(BeautifulSoup(text, "html.parser").get_text())

    news_items = []
    
    for item in items:
        title = clean(item.get("title", "제목 없음"))
        desc = clean(item.get("description", ""))
        link = item.get("originallink") or item.get("link") or f"https://search.naver.com/search.naver?query={query}"
        pub_date = item.get("pubDate", "날짜 없음")  # 기사 발행 날짜
        
        # 발행 날짜 형식 변환
        try:
            pub_date = datetime.strptime(pub_date, '%a, %d %b %Y %H:%M:%S %z').strftime('%Y-%m-%d %H:%M:%S')
        except Exception as e:
            pub_date = "날짜 변환 오류"  # 날짜 형식 오류 시 기본 메시지
            
        # 결과에 제목, 링크, 설명, 날짜 추가
        news_items.append({
            "제목": title,
            "내용": desc,
            "링크": link,
            "발행 날짜": pub_date  # 날짜 형식 추가
        })
    
    return news_items


In [23]:

# 실행 예시
company_name = "테슬라"  # 검색할 기업 이름
with no_ssl_verification():
    news = get_news_list(company_name, display= 100)

# 결과 출력
if news:
    for idx, article in enumerate(news, 1):
        print(f"{idx}. 제목: {article['제목']}")
        print(f"   내용: {article['내용']}")
        print(f"   링크: {article['링크']}")

API 호출 성공!
1. 제목: 삼성전자, 파운드리 적자 내년까지 길어질 전망
   내용: 테슬라향 수주에도, 종합 반도체 기업으로서 삼성전자가 겪는 고객사와의 이해상충 문제는 여전한... 반대로 테슬라가 전기차용 자율주행칩 제조를 삼성전자에 맡긴 데는 TSMC에 의존하고 있는 칩 공급선 다변화... 
   링크: https://www.straightnews.co.kr/news/articleView.html?idxno=277640
2. 제목: 테슬라 삼성전자와 반도체 협력에도 성과 불투명, 구글 웨이모 추격 어...
   내용: 테슬라가 삼성전자와 대규모 반도체 파운드리 계약을 비롯해 자율주행 기술 발전에 공격적으로 투자를 늘리고 있지만 구글 웨이모 등 경쟁사와 여전히 격차를 보인다는 지적이 나왔다. 테슬라 전기차에 적용되는... 
   링크: https://www.businesspost.co.kr/BP?command=article_view&num=406124
3. 제목: 상반기 전기차 배터리 37% 증가… 중국 독주, 한국은 부진
   내용: LG에너지솔루션은 47.2GWh(점유율 9.4%)로 전년보다 4.4% 증가하며 3위를 유지했지만, 테슬라향 물량이 줄어든 것이 성장세를 제약했다. 반면, 기아 EV3와 GM의 얼티엄 플랫폼 기반 모델 판매 호조가 일부 만회... 
   링크: https://www.thepowernews.co.kr/view.php?ud=202508041053206970de3f0aa1be_7
4. 제목: 글로벌 전기차 배터리 수요 급증…한국 3사 점유율 5%p 하락
   내용: LG에너지솔루션은 47.2GWh를 기록해 글로벌 3위를 유지했으나, 테슬라향 물량 감소 영향으로 성장폭이 4.4%에 그쳤다. 테슬라 모델 판매 부진으로 LG엔솔의 테슬라향 배터리 사용량은 28.9% 급감했다. SK온은 19.... 
   링크: https://it.chosun.com/news/articleView.html?idxno=2023092145186
5. 제

In [24]:
news_df = pd.DataFrame(news)

In [25]:

# 4. 감성 분석을 통해 점수 부여
def analyze_sentiment(text):
    result = pipe(text)  # 모델로 텍스트 분석
    label = result[0]['label']  # '긍정', '부정', '중립' 중 하나
    score = result[0]['score']  # 감성 점수
    return label, score

# 5. 감성 분석 결과를 데이터프레임에 추가
news_df[['감성', '감성 점수']] = news_df['내용'].apply(lambda x: pd.Series(analyze_sentiment(x)))

c:\python_work\gpt_agent_2025\myenv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [28]:
news_df.head(10)

,제목,내용,링크,발행 날짜,감성,감성 점수
0,"삼성전자, 파운드리 적자 내년까지 길어질 전망","테슬라향 수주에도, 종합 반도체 기업으로서 삼성전자가 겪는 고객사와의 이해상충 문제...",https://www.straightnews.co.kr/news/articleVie...,2025-08-04 11:00:00,negative,0.998574
1,"테슬라 삼성전자와 반도체 협력에도 성과 불투명, 구글 웨이모 추격 어...",테슬라가 삼성전자와 대규모 반도체 파운드리 계약을 비롯해 자율주행 기술 발전에 공격...,https://www.businesspost.co.kr/BP?command=arti...,2025-08-04 10:58:00,positive,0.992056
2,"상반기 전기차 배터리 37% 증가… 중국 독주, 한국은 부진",LG에너지솔루션은 47.2GWh(점유율 9.4%)로 전년보다 4.4% 증가하며 3위...,https://www.thepowernews.co.kr/view.php?ud=202...,2025-08-04 10:56:00,positive,0.998979
3,글로벌 전기차 배터리 수요 급증…한국 3사 점유율 5%p 하락,"LG에너지솔루션은 47.2GWh를 기록해 글로벌 3위를 유지했으나, 테슬라향 물량 ...",https://it.chosun.com/news/articleView.html?id...,2025-08-04 10:56:00,negative,0.961156
4,"테슬라, 오토파일럿 사망 사건으로 3300억원 배상 책임…자율주행 산업...",미국 법원이 테슬라의 자율주행 기술에 결함이 있다는 판단을 내렸다. 이에 따라 미국...,https://www.epnc.co.kr/news/articleView.html?i...,2025-08-04 10:50:00,negative,0.999659
5,테슬라 식당 개점 고작 2주 지났는데…주민 불만 폭발,테슬라의 복합 다이너 공간 [사진: 테슬라라티(Teslarati)] 테슬라가 7월 ...,https://www.digitaltoday.co.kr/news/articleVie...,2025-08-04 10:48:00,neutral,0.999265
6,"SK온-엔무브 합병, 약속의 '2026년' 상장 시도할까","LG에너지솔루션은 최근 테슬라 ESS향으로 추정되는 6조원 대 계약을 발표했으며, ...",http://www.smedaily.co.kr/news/articleView.htm...,2025-08-04 10:48:00,positive,0.999812
7,트럼프 '슈퍼팩'에 가상자산 기업 응답했다...전체 모금액 20% 육박,한편 마가는 억만장자 투자자 제프 야스와 일론 머스크 테슬라 CEO 등도 주요 후원...,https://www.techm.kr/news/articleView.html?idx...,2025-08-04 10:44:00,neutral,0.990485
8,"LG·SK·삼성 韓 3사, 상반기 세계 전기차 배터리 점유율 5.4%p↓…중국 ...","LG에너지솔루션의 배터리 사용량은 주로 테슬라, 쉐보레, 기아, 폭스바겐 등의 주요...",https://www.getnews.co.kr/news/articleView.htm...,2025-08-04 10:44:00,negative,0.999503
9,삼성·애플 팔리면 IT 주가도 뜰까···테슬라 AI도 변수,또한 삼성전자와 테슬라 간 약 22조8000억원 규모의 인공지능 반도체(AI6) 공...,https://www.womaneconomy.co.kr/news/articleVie...,2025-08-04 10:44:00,positive,0.999863


In [33]:
news_df.groupby('감성').describe().T

감성            negative    neutral   positive
감성 점수 count  31.000000  26.000000  43.000000
      mean    0.954058   0.969948   0.952278
      std     0.092490   0.068707   0.104342
      min     0.632964   0.661961   0.431358
      25%     0.960706   0.972818   0.973261
      50%     0.998804   0.997136   0.993892
      75%     0.999483   0.998637   0.999775
      max     0.999784   0.999907   0.999875

In [34]:
news_df.to_clipboard()

In [35]:
from curl_cffi.requests import Session
import yfinance as yf
import pandas as pd
import numpy as np
import ffn

In [40]:
session = Session(
        impersonate="chrome110",  # optional, mimics Chrome browser
        headers={
            "User-Agent": "Mozilla/5.0"
        },
        verify=False  # optional; disable SSL verify if you're behind a proxy
    )

In [41]:
def get_yf_stock_info(ticker: str) -> str:
    """해당 종목의 Yahoo Finance 정보를 반환합니다."""
    stock = yf.Ticker(ticker, session = session)
    info = stock.info
    return str(info)

In [42]:
get_yf_stock_info('MSFT')

"{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power apps, and po

In [4]:
import praw
import ssl
import pandas as pd
from dotenv import load_dotenv
import os

# 환경 변수 로드
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# SSL 인증 우회
ssl._create_default_https_context = ssl._create_unverified_context

# Reddit 인증 정보
reddit_client_id = "RVKUBtrh7ExzRSbddfBDtg"
reddit_client_secret = "cT4m_YrJnZhZpZ2vPkVTfMT8hqj07A"
reddit_user_agent = "retail_stock_v1.0 (by /u/TraditionalIce9098)"

# Reddit 클라이언트 생성
reddit = praw.Reddit(
    client_id=reddit_client_id,
    client_secret=reddit_client_secret,
    user_agent=reddit_user_agent,
    check_for_async=False
)

# 인증 우회
reddit._core._requestor._http.verify = False

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)
def extract_related_tickers_gpt(title: str, body: str) -> list:
    prompt = f"""
다음 텍스트에서 언급된 주요 기업명 또는 주식 티커(symbol)를 추출하세요.
미국 주식 기준으로 반환하고, 없으면 빈 리스트로.

텍스트:
"{title}
{body}"

결과는 JSON 배열로 반환하세요. 예: ["AAPL", "TSLA"]
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            temperature=0.2,
            messages=[
                {"role": "system", "content": "당신은 금융 분석에 능한 주식 전문가입니다."},
                {"role": "user", "content": prompt}
            ]
        )
        content = response.choices[0].message.content
        tickers = eval(content.strip()) if "[" in content else []
        return tickers
    except Exception as e:
        print("❗ GPT 추출 오류:", e)
        return []

In [5]:
def get_reddit_top_posts(subreddit_name: str, limit: int = 30) -> pd.DataFrame:
    subreddit = reddit.subreddit(subreddit_name)
    posts = subreddit.hot(limit=limit)

    data = []
    for post in posts:
        title = post.title
        content = post.selftext.strip()
        tickers = extract_related_tickers_gpt(title, content)
        related = ", ".join(tickers) if tickers else ""

        data.append({
            "제목": title,
            "내용": content,
            "관련종목": related
        })

    return pd.DataFrame(data)

In [6]:
posts = get_reddit_top_posts("stocks", limit=10)
posts

c:\python_work\gpt_agent_2025\myenv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.reddit.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\python_work\gpt_agent_2025\myenv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oauth.reddit.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


❗ GPT 추출 오류: invalid syntax (<string>, line 1)


,제목,내용,관련종목
0,Rate My Portfolio - r/Stocks Quarterly Thread ...,Please use this thread to discuss your portfol...,
1,/r/Stocks Weekend Discussion Saturday - Aug 02...,This is the weekend edition of our stickied di...,SPY
2,Will we retroactively be declared to have alre...,The National Bureau of Economic Research (NBER...,
3,Best stocks to buy after the tariffs,I’m hoping to buy the dip for companies that w...,
4,Is Figma today’s Beyond Meat IPO?,"In 2019 Beyond Meat was oversubscribed, same a...","Figma, Beyond Meat"
5,Waller and Bowman say waiting to cut rates thr...,[Source CNBC](https://www.cnbc.com/2025/08/01/...,
6,Figma is not Ai - its a industry standard prot...,I've seen way too many posts and ignorant comm...,"Figma, Adobe"
7,"After recent jobs data, Moody's model raises r...","Moody's forecast model for recession, which ha...",
8,$NVT still good to get in?,Was eyeing nVent electric this past month as I...,"NVT, NVDA"
9,"MOAT, VOO or picking on my own",So after 2.5 years of trying different methods...,"MOAT, VOO, FICO"


In [7]:
import praw
from openai import OpenAI
from dotenv import load_dotenv
import os
from no_ssl_verification import no_ssl_verification
# .env 파일에서 환경 변수 로드
load_dotenv()

import pandas as pd
import os
import numpy as np
import openai
import json 
import requests
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Reddit 인증 정보
reddit_client_id = "RVKUBtrh7ExzRSbddfBDtg"
reddit_client_secret = "cT4m_YrJnZhZpZ2vPkVTfMT8hqj07A"
reddit_user_agent = "retail_stock_v1.0 (by /u/TraditionalIce9098)"

# Reddit 클라이언트 생성
reddit = praw.Reddit(
    client_id=reddit_client_id,
    client_secret=reddit_client_secret,
    user_agent=reddit_user_agent,
    check_for_async=False
)

In [52]:
# 예시: Reddit subreddit에서 글 가져오기
with no_ssl_verification():
    subreddit = reddit.subreddit("wallstreetbets")
    for submission in subreddit.hot(limit=5):
        print(submission.title)

What Are Your Moves Tomorrow, August 04, 2025
Weekly Earnings Thread 8/4 - 8/8
Stay strong UNH bagholders
$UNH is the most mentioned stock in this subreddit
Not stopping until we reach half a mili


In [8]:
# 🔍 submission -> dict로 변환
def parse_submission(submission):
    return {
        'title': submission.title,
        'created_utc': submission.created_utc,
        'author_fullname': getattr(submission, 'author_fullname', None),
        'selftext': submission.selftext,
        'score': submission.score,
        'url': submission.url,
        'num_comments': submission.num_comments,
        'id': submission.id
    }

# 🔄 게시물 수집
def fetch_wallstreetbets_posts(limit=1000):
    with no_ssl_verification():
        subreddit = reddit.subreddit("wallstreetbets")
        posts = []
        count = 0

        for submission in subreddit.hot(limit=limit):  # 또는 .new(), .top(), .rising() 가능
            try:
                post_data = parse_submission(submission)
                posts.append(post_data)
                count += 1

                if count % 100 == 0:
                    print(f"✅ 수집된 게시글 수: {count}")
            except Exception as e:
                print(f"❌ 오류 발생: {e}")
                continue

    return posts

In [9]:
posts = fetch_wallstreetbets_posts(limit=10)

In [10]:
import openai
import json
import os
import time

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def analyze_reddit_post(post: dict) -> dict:
    """
    Reddit 게시물(title, selftext, image 포함)을 GPT-4o로 종합 분석하여 결과 JSON 리턴
    """
    title = post.get("title", "")
    body = post.get("selftext", "")
    image_url = post.get("image_url", "")
    
    # 이미지 분석 (있을 경우)
    if image_url:
        img_prompt = "reddit 의 wallstreetbets에 나온 투자 관련 이미지야. 이 관점에서 이미지가 어떤 내용일지 해석해줘"
        try:
            img_response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": img_prompt},
                            {"type": "image_url", "image_url": {"url": image_url}},
                        ]
                    }
                ]
            )
            image_analysis = img_response.choices[0].message.content.strip()
        except Exception as e:
            print(f"이미지 분석 오류: {e}")
            image_analysis = "이미지 없음"
    else:
        image_analysis = "이미지 없음"

    # 텍스트 기반 요약 및 종목 추출
    system_prompt = {
        "role": "system",
        "content": """
        reddit 의 wallstreetbets에 나온 투자 관련글이야.
        제목 / 본문 / 이미지를 조합해서 해당 글이 어떤 내용인지 한국어로 요약해줘.
        결과물은 아래와 같은 json 양식으로 써줘:
        {
          "구분": "투자정보 / 투자후기 등",
          "내용요약": "~~ 가 좋은 종목이다 / ~~를 투자해서 돈을 벌었다 등",
          "재미요소": "작성자가 큰 금액을 날린것을 희화화함 or 언어유희 등",
          "관련종목": "META (해당 종목의 티커)"
        }
        """
    }

    user_prompt = {
        "role": "user",
        "content": f"컨텐츠 제목 : {title} / 컨텐츠 본문 : {body} / 이미지 : {image_analysis}"
    }

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[system_prompt, user_prompt],
            temperature=0
        )
        content = response.choices[0].message.content.strip()
        res_json = json.loads(content.replace('```json','').replace('```',''))
        return res_json
    except Exception as e:
        print(f"본문 분석 오류: {e}")
        return {
            "구분": "분석실패",
            "내용요약": "",
            "재미요소": "",
            "관련종목": ""
        }

In [ ]:
from tqdm import tqdm

enriched_posts = []

for post in tqdm(posts):
    result = analyze_reddit_post(post)  # 여기에서 post를 넣어야 함
    post.update({
        "구분": result.get("구분", ""),
        "내용요약": result.get("내용요약", ""),
        "재미요소": result.get("재미요소", ""),
        "related_tickers": result.get("관련종목", "").split(",") if result.get("관련종목") else []
    })
    enriched_posts.append(post)

100%|██████████| 10/10 [00:29<00:00,  2.98s/it]


: 

In [1]:
pd.DataFrame(enriched_posts).to_clipboard()

NameError: name 'pd' is not defined